# Setup and Load Data

***

In [ ]:
!pip install tensorflow opencv-python matplotlib

In [ ]:
!pip list

In [ ]:
import tensorflow as tf
import os

In [ ]:
gpus = tf.config.experimental.list_physical_devices("GPU")
print(gpus)

In [ ]:
#To avoid OOM(Out Of Memory) error, set GPU to certain Memory Consumption Growth
gpus = tf.config.experimental.list_physical_devices("GPU")
for gpu in gpus:
    tf.config.experimental.set_memory_growth(gpu,True)

***

# Remove dodgy images

***

In [ ]:
import cv2
import imghdr
from matplotlib import pyplot as plt

In [ ]:
data_directory_path = 'data'

In [ ]:
os.listdir(os.path.join(data_directory_path,'happy'))

In [ ]:
image_extension = ['jpeg','jpg','bmp','png']

In [ ]:
image = cv2.imread(os.path.join('data','happy','smile.woman_.jpg'))

In [ ]:
image.shape #heght, width, color(if color = 3:Colored, else:b&w)

In [ ]:
plt.imshow(cv2.cvtColor(image, cv2.COLOR_BGR2RGB))
plt.show()

In [ ]:
for image_class in os.listdir(data_directory_path):
    for image in os.listdir(os.path.join(data_directory_path, image_class)):
        image_path = os.path.join(data_directory_path, image_class, image)
        try:
            img = cv2.imread(image_path)
            tip = imghdr.what(image_path)
            if tip not in image_extension:
                print('Image not in ext list {}'.format(image_path))
                os.remove(image_path)
        except Exception as e:
            print('Issue with image {},{}'.format(image_path))

***

# Load Data

***

In [ ]:
tf.data.Dataset??

In [ ]:
import numpy as np

In [ ]:
data = tf.keras.utils.image_dataset_from_directory('data')

Two classes are created one for happy people and one class for sad.

In [ ]:
data_iterator = data.as_numpy_iterator()

In [ ]:
#Get another batch from the iterator
batch = data_iterator.next()

In [ ]:
len(batch) #Image Set and Label Set respectively will be batch, which can be accessed through indexing

In [ ]:
#Images represented as numpy arrays
#Image dataset is divided into batches of 32 images by keras
batch[0].shape

In [ ]:
#The labels are represented as 1 and 0.
#Class 0 : Happy
#Class 1 : Sad
#For every iteration new batch 32 images is pushed throug the pipeline.
batch[1] 

In [ ]:
fig, ax = plt.subplots(ncols=4, figsize=(20,20))
for idx, img in enumerate(batch[0][:4]):
    ax[idx].imshow(img.astype(int)) 
    #Converts the image data type to int for display purposes.
    #converts the pixel values of the image (img) to integers. 
    #This assumes that the original pixel values are already in a normalized range (e.g., 0-1) and multiplies them by 255 to scale them to the 0-255 range.
    ax[idx].title.set_text(batch[1][idx]) #Here scaled_batch[1] represents the label set.

***

# Preprocess Data

***

#### Scale Data = When we load the images, the values of each image is between 0 and 255. For optimizing we can scale the image's vale between 0 and 1.

In [ ]:
batch[0].min() 

In [ ]:
data = data.map(lambda x,y: (x/255, y))

In [ ]:
scaled_iterator = data.as_numpy_iterator()

In [ ]:
scaled_batch = next(scaled_iterator)

In [ ]:
fig, plot = plt.subplots(ncols=4, figsize=(20,20))
for indx, img in enumerate(scaled_batch[0][:4]):
    plot[indx].imshow(img) #Here conversion to int is not possible as the scaling is done between 0 and 1.
    plot[indx].title.set_text(scaled_batch[1][indx]) #Here scaled_batch[1] represents the label set.

##### Split data for training, validation and testing

In [ ]:
# length of data(number of batches)
len(data)

In [ ]:
train_size = int(len(data)*.7) #70% of batches
val_size = int(len(data)*.2) #20% of batches
test_size = int(len(data)*.1)+1 #10% of batches

In [ ]:
train_size + val_size + test_size #total number of batches

In [ ]:
#creating the data according to the above mentioned sizes.
train = data.take(train_size)
val = data.skip(train_size).take(val_size) #skip() method skips the alreay taken batches
test = data.skip(train_size + val_size).take(test_size)

##### Building Deep Learning models

In [ ]:
from tensorflow.keras.models import Sequential # Keras Sequential Model 
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Dense, Flatten # Layers added to the model

In [ ]:
model = Sequential()

In [ ]:
# Layers are added one by one to the model in the sequential manner

model.add(Conv2D(16,(3,3),1, activation='relu', input_shape=(256,256,3)))
model.add(MaxPooling2D())

model.add(Conv2D(32,(3,3),1, activation='relu'))
model.add(MaxPooling2D())

model.add(Conv2D(16,(3,3),1, activation='relu'))
model.add(MaxPooling2D())

model.add(Flatten())

model.add(Dense(256, activation='relu'))
model.add(Dense(1, activation='sigmoid')) #Single Output Layer 
# Here the sigmoid activation brings the value between 0 and 1, which is used for detection of sad and happy people.

In [ ]:
model.compile('adam',loss=tf.losses.BinaryCrossentropy(), metrics=['accuracy']) #using optimazation module adam

In [ ]:
model.summary()

##### Training the model

In [ ]:
logdir = 'logs'

In [ ]:
tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir=logdir)

In [ ]:
# Training Component
# Epochs specify the number of runs over the training data set. If it is 1, its going to be 1 run over the training dataset.
hist = model.fit(train, epochs=20, validation_data=val, callbacks=[tensorboard_callback])

##### Plot Performance

In [ ]:
fig = plt.figure()
plt.plot(hist.history['loss'], color='teal', label='loss')
plt.plot(hist.history['val_loss'], color='orange', label='val_loss')
fig.suptitle('Loss', fontsize=20)
plt.legend(loc='upper left')
plt.show()

In [ ]:
fig = plt.figure()
plt.plot(hist.history['accuracy'], color='teal', label='accuracy')
plt.plot(hist.history['val_accuracy'], color='orange', label='val_accuracy')
fig.suptitle('Accuracy', fontsize=20)
plt.legend(loc='upper left')
plt.show()

# Evaluate Performance

##### Evaluate

In [ ]:
# Importing measures for this classification problem
from tensorflow.keras.metrics import Precision, Recall, BinaryAccuracy

In [ ]:
# Creating instances for measures
pre = Precision()
rec = Recall()
accu = BinaryAccuracy()

In [ ]:
for scaled_batch in test.as_numpy_iterator():
    X,y = scaled_batch # X= set of images
    yhat = model.predict(X) # yhat value will be either 0 or 1, its predicted, as of the images and that is done through sigmoid activation
    pre.update_state(y, yhat) # Here the comparison is done between y and yhat value, where y stores the orginal label of the image and yhat is the newly predcted one.
    rec.update_state(y, yhat)
    accu.update_state(y, yhat)

In [ ]:
print(f'Precision:{pre.result().numpy()}, Recall:{rec.result().numpy()}, Accuracy:{accu.result().numpy()}')

##### Test

In [ ]:
img = cv2.imread('sad_image_test.jpg')
plt.imshow(cv2.cvtColor(img, cv2.COLOR_BGR2RGB))
plt.show()

In [ ]:
# For the neural network every image must be (256px Hight,256px Width, 3 Chanells(RGB))
resize = tf.image.resize(img, (256, 256))
plt.imshow(resize.numpy().astype(int))
plt.show()

In [ ]:
#Passing the image thorugh the neural network
yhat = model.predict(np.expand_dims(resize/255, 0)) 

In [ ]:
yhat

In [ ]:
if yhat > 0.5 :
    print("The preidicted image is SAD")
else:
    print("The predicted image is HAPPY")

# Saving the Model

In [ ]:
from tensorflow.keras.models import load_model

In [ ]:
model.save(os.path.join('Models','Happy_or_Sad_Image-Classification.h5'))

In [ ]:
#Loading the model
new_model = load_model(os.path.join('Models','Happy_or_Sad_Image-Classification.h5'))

In [ ]:
# Passing the resized image through the model
new_prediction = new_model.predict(np.expand_dims(resize/255, 0))

In [ ]:
if new_prediction > 0.5:
    print("The predicted image is SAD")
else:
    print("The predicted image is HAPPY")